In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, kstest, norm
import chardet
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from statsmodels.graphics.gofplots import qqplot

INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "check_normality"
GRAPHS_FOLDER = os.path.join(RESULTS_FOLDER, "graphs_of_distributions")
for folder in [RESULTS_FOLDER, GRAPHS_FOLDER]:
    os.makedirs(folder, exist_ok=True)

sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

def detect_encoding(file_path):
    """Визначає кодування файлу."""
    try:
        with open(file_path, 'rb') as f:
            return chardet.detect(f.read(10000))['encoding'] or 'utf-8'
    except Exception as e:
        print(f"Помилка визначення кодування для {file_path}: {e}")
        return 'utf-8'

def detect_separator(file_path, encoding):
    """Визначає роздільник у CSV-файлі."""
    try:
        with open(file_path, encoding=encoding) as f:
            sample = f.read(2048)
            return csv.Sniffer().sniff(sample).delimiter
    except Exception as e:
        print(f"Помилка визначення роздільника для {file_path}: {e}")
        return ','

def load_csv(file_path):
    """Завантажує CSV-файл."""
    encoding = detect_encoding(file_path)
    sep = detect_separator(file_path, encoding)
    print(f"   Кодування: {encoding}, роздільник: '{sep}'")
    
    try:
        df = pd.read_csv(file_path, encoding=encoding, sep=sep, quotechar='"', on_bad_lines='skip')
        print(f"   Перші 5 колонок: {list(df.columns[:5])}")
        return df
    except Exception as e:
        print(f"Помилка читання {file_path}: {e}")
        return None

def detect_dataset_type(filename):
    """Визначає тип датасету за назвою файлу."""
    match = re.search(r'(\d{4})', filename)
    if match:
        year = int(match.group(1))
        return 'NMT' if year >= 2022 else 'ZNO'
    return None

def check_normality(df, dataset_type, filename, normality_results):
    """Перевіряє нормальність розподілу середньої оцінки."""
    score_columns = [col for col in df.columns if col.endswith('_score_100')]
    if not score_columns:
        print(f"⚠️ Для {filename} не знайдено колонок з оцінками (*_score_100)")
        return
    
    df['average_score'] = df[score_columns].mean(axis=1, skipna=True)
    scores = df['average_score'].dropna()
    
    if len(scores) < 3:
        print(f"⚠️ Недостатньо даних для перевірки нормальності в {filename}")
        return
    
    ks_stat, ks_p = kstest(scores, 'norm', args=(scores.mean(), scores.std()))
  
    normality_results.append({
        'file': filename,
        'dataset_type': dataset_type,
        'sample_size': len(scores),
        'ks_stat': ks_stat,
        'ks_p_value': ks_p,
    })
    
    plt.figure(figsize=(10, 6))
    sns.histplot(scores, kde=False, stat='density', bins=30, color='#36A2EB', edgecolor='black')
    x = np.linspace(scores.min(), scores.max(), 100)
    plt.plot(x, norm.pdf(x, scores.mean(), scores.std()), 'r-', lw=2, label='Нормальний розподіл')
    plt.title(f'Розподіл середньої оцінки ({dataset_type}, {filename})', pad=20)
    plt.xlabel('Середня оцінка')
    plt.ylabel('Густота')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(GRAPHS_FOLDER, f'normality_hist_{filename}.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(10, 6))
    qqplot(scores, line='s', marker='o', markerfacecolor='#FF6384', markeredgecolor='black', alpha=0.5)
    plt.title(f'Q-Q плот середньої оцінки ({dataset_type}, {filename})', pad=20)
    plt.xlabel('Теоретичні квантилі')
    plt.ylabel('Емпіричні квантилі')
    plt.tight_layout()
    plt.savefig(os.path.join(GRAPHS_FOLDER, f'qq_plot_{filename}.png'), dpi=300, bbox_inches='tight')
    plt.close()

normality_results = []


for filename in os.listdir(INPUT_FOLDER):
    if not filename.endswith(".csv"):
        continue
    
    path = os.path.join(INPUT_FOLDER, filename)
    print(f"🔄 Обробка {filename}...")
    
    try:
        df = load_csv(path)
        if df is None or df.empty:
            print(f"⚠️ Файл {filename} порожній або не вдалося завантажити")
            continue
        
        dataset_type = detect_dataset_type(filename)
        if not dataset_type:
            print(f"⚠️ Невідомий тип даних: {filename}")
            continue
        
        print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")
        
        check_normality(df, dataset_type, filename, normality_results)
        
    except Exception as e:
        print(f"❌ Помилка у файлі {filename}: {e}")
        import traceback
        traceback.print_exc()

pd.DataFrame(normality_results).to_csv(os.path.join(RESULTS_FOLDER, 'normality_tests.csv'), index=False)
print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")

🔄 Обробка 2020.csv...
   Кодування: utf-8, роздільник: ','


/var/folders/gh/9jz8pwfn5cv995vfm04sql_40000gn/T/ipykernel_3968/217023426.py:52: DtypeWarning: Columns (96,97,100,102,103,104,105,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding=encoding, sep=sep, quotechar='"', on_bad_lines='skip')


   Перші 5 колонок: ['id', 'birth_year', 'gender', 'region_name', 'area_name']
   Завантажено 201212 рядків, 131 стовпців
🔄 Обробка 2021.csv...
   Кодування: utf-8, роздільник: ','


/var/folders/gh/9jz8pwfn5cv995vfm04sql_40000gn/T/ipykernel_3968/217023426.py:52: DtypeWarning: Columns (117,118,121,123,124,125,126,137,138,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding=encoding, sep=sep, quotechar='"', on_bad_lines='skip')


   Перші 5 колонок: ['id', 'birth_year', 'gender', 'region_name', 'area_name']
   Завантажено 188609 рядків, 152 стовпців
🔄 Обробка 2023.csv...
   Кодування: utf-8, роздільник: ','
   Перші 5 колонок: ['id', 'birth_year', 'gender', 'region_name', 'area_name']
   Завантажено 256313 рядків, 64 стовпців
🔄 Обробка 2022.csv...
   Кодування: utf-8, роздільник: ','


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>